In [2]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession

spark_session = (configure_spark_with_delta_pip(SparkSession.builder.master("local[*]")
                                                        .config("spark.sql.catalogImplementation", "hive")
                                                .config("spark.sql.extensions",
                                                        "io.delta.sql.DeltaSparkSessionExtension")
                                                .config("spark.sql.catalog.spark_catalog",
                                                        "org.apache.spark.sql.delta.catalog.DeltaCatalog")
                                                ).getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/23 06:23:04 WARN Utils: Your hostname, bartosz, resolves to a loopback address: 127.0.1.1; using 192.168.1.55 instead (on interface wlp0s20f3)
25/08/23 06:23:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/bartosz/.venvs/delta_spark_4/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/bartosz/.ivy2.5.2/cache
The jars for the packages stored in: /home/bartosz/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5c595551-a2a8-4c3a-bfb6-babc24e8a34d;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in local-m2-cache
:: resolution report :: resolve 167ms :: artifacts dl 6ms


**Create input tables first**

In [3]:
rm -rf ./spark-warehouse && rm -rf ./metastore_db/

In [4]:
tables = ['table_1', 'table_2']
for table in tables:
    print(f'Creating {table}')
    spark_session.sql(f'DROP TABLE IF EXISTS `default`.`{table}`')
    spark_session.sql(f'''
              CREATE TABLE `default`.`{table}` (
                 number INT NOT NULL,
                letter STRING NOT NULL
              ) USING DELTA
            ''')


Creating table_1


25/08/23 06:23:14 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/08/23 06:23:14 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore bartosz@127.0.1.1
25/08/23 06:23:14 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
25/08/23 06:23:17 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`table_1` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/08/23 06:23:17 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/08/23 06:23:17 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist


Creating table_2


25/08/23 06:23:18 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`table_2` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


# Reverse proxy

Let's start with writing some rows to the first of two tables involves in the dual write:

In [5]:
from pyspark import Row

data_to_write = spark_session.createDataFrame([Row(number=1, letter='a'), Row(number=2, letter='b'), Row(number=3, letter='c')])

data_to_write.write.format('delta').mode('overwrite').insertInto('table_1')
spark_session.read.table('table_1').show()

25/08/23 06:23:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+------+
|number|letter|
+------+------+
|     3|     c|
|     1|     a|
|     2|     b|
+------+------+



Let's now suppose we want to write the same dataset to the *table_2* but the writer fails with some random exception.

In [6]:
raise RuntimeError('Some error occurred')

RuntimeError: Some error occurred

If we need to apply the **`Proxy`** with Delta Lake, we need to create the views with most recent commit versions for both tables only when the writes to both tables succeeded. As this approach requires some extra table to track the last successful commits per table, let's move directly to the wrap-up part where you will find a more complete example.  

# Wrap-up
Let's wrap the *Proxy* into a more realistic code. We create here two tables involved in the dual write and a mapping table:

In [7]:
tables = ['table_3', 'table_4']
for table in tables:
    print(f'Creating {table}')
    spark_session.sql(f'DROP TABLE IF EXISTS `default`.`{table}`')
    spark_session.sql(f'''
              CREATE TABLE `default`.`{table}` (
                 number INT NOT NULL,
                letter STRING NOT NULL
              ) USING DELTA
            ''')

spark_session.sql(f'DROP TABLE IF EXISTS `default`.`last_versions_per_table`')
spark_session.sql(f'''
          CREATE TABLE `default`.`last_versions_per_table` (
             version INT NOT NULL,
             table_name STRING NOT NULL
          ) USING DELTA
        ''')

Creating table_3


25/08/23 06:27:46 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`table_3` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


Creating table_4


25/08/23 06:27:47 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`table_4` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/08/23 06:27:47 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`last_versions_per_table` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [8]:
def get_last_version(table_name: str) -> int:
    spark_session.sql(f'DESCRIBE HISTORY {table_name}').createOrReplaceTempView('history')
    versions_df = spark_session.sql(f'SELECT MAX(version) AS version FROM history ORDER BY version DESC LIMIT 2').collect()
    return versions_df[0].version


def get_all_versions_from_the_last_versions_per_table():
    versions = spark_session.sql('SELECT table_name, version FROM last_versions_per_table').collect()
    versions_dict = {row.table_name: row.version for row in versions}
    return versions_dict

def update_last_versions_in_last_versions_per_table(tables_with_versions: list[(str, int)]):
    print(tables_with_versions)
    updates = [Row(table_name=table_version[0], version=table_version[1]) for table_version in tables_with_versions.items()]
    update_to_write = spark_session.createDataFrame(updates)
    (DeltaTable.forName(spark_session, 'last_versions_per_table').alias('base_table')
     .merge(update_to_write.alias('new_table'), 'base_table.table_name = new_table.table_name')
     .whenMatchedUpdateAll().whenNotMatchedInsertAll().execute())

def create_view(table_for_proxy: str, tables_with_versions: dict[str, int]):
    version_to_use = 0
    if table_for_proxy in tables_with_versions:
        version_to_use = tables_with_versions[table_for_proxy]
    view_name = f'view_{table_for_proxy}'
    spark_session.sql(f'DROP VIEW IF EXISTS {view_name}')
    print(f'Creating view {view_name} from table {table_for_proxy} and version {version_to_use}')
    spark_session.sql(f'''CREATE VIEW {view_name} AS 
        (SELECT * FROM {table_for_proxy} VERSION AS OF {version_to_use})''')


In [9]:
from pyspark import Row

def demo(should_fail: bool):
    data_to_write = spark_session.createDataFrame([Row(number=1, letter='a'), Row(number=2, letter='b'), Row(number=3, letter='c')])
    
    tables_with_versions = {}
    
    def write_data(table: str) -> bool:
        try:
            if should_fail and table == 'table_4':
                print('Error!')
                raise RuntimeError('Some random error')
            data_to_write.write.format('delta').mode('overwrite').insertInto(table)
            tables_with_versions[table] = get_last_version(table)
            return True
        except Exception as error:
            print(error)
            return False
    
    result_1 = write_data('table_3')
    result_2 = write_data('table_4')
    print(f'result_1={result_1} // result_2={result_2}')
    if result_1 and result_2:
        print(tables_with_versions)
        update_last_versions_in_last_versions_per_table(tables_with_versions)
        all_versions = tables_with_versions
    else:
        all_versions = get_all_versions_from_the_last_versions_per_table()
    
    create_view('table_3', all_versions)
    create_view('table_4', all_versions)
    print(all_versions)
    spark_session.read.table('view_table_3').show()
    spark_session.read.table('view_table_4').show()

demo(True)

Error!
Some random error
result_1=True // result_2=False


Creating view view_table_3 from table table_3 and version 0
Creating view view_table_4 from table table_4 and version 0
{}
+------+------+
|number|letter|
+------+------+
+------+------+

+------+------+
|number|letter|
+------+------+
+------+------+



Let's see now what happens if we have some committed entries in the last_versions_per_table table. Let's create a successful write first:

In [10]:
demo(False)

result_1=True // result_2=True
{'table_3': 2, 'table_4': 1}
{'table_3': 2, 'table_4': 1}


25/08/23 06:29:18 WARN MapPartitionsRDD: RDD 281 was locally checkpointed, its lineage has been truncated and cannot be recomputed after unpersisting


Creating view view_table_3 from table table_3 and version 2
Creating view view_table_4 from table table_4 and version 1
{'table_3': 2, 'table_4': 1}
+------+------+
|number|letter|
+------+------+
|     1|     a|
|     2|     b|
|     3|     c|
+------+------+

+------+------+
|number|letter|
+------+------+
|     3|     c|
|     2|     b|
|     1|     a|
+------+------+



And now, let's suppose the dual write fails. The view shouldn't be upgraded to the partially committed tables:

In [11]:
demo(True)

Error!
Some random error
result_1=True // result_2=False
Creating view view_table_3 from table table_3 and version 2
Creating view view_table_4 from table table_4 and version 1
{'table_4': 1, 'table_3': 2}
+------+------+
|number|letter|
+------+------+
|     1|     a|
|     2|     b|
|     3|     c|
+------+------+

+------+------+
|number|letter|
+------+------+
|     3|     c|
|     2|     b|
|     1|     a|
+------+------+

